NOTE: redoo p-value when all samples are made. P-values before that are not real but would prob be close enough for plotting and debugging

In [1]:
from pg import DB
db = DB()

In [50]:
sample_average_score_sizes = db.query("Select Distinct size from samples_scores order by size desc").getresult()
sample_average_score_sizes

[(334,),
 (333,),
 (332,),
 (331,),
 (330,),
 (329,),
 (328,),
 (327,),
 (326,),
 (325,),
 (324,),
 (323,),
 (322,),
 (321,),
 (320,),
 (319,),
 (318,),
 (317,),
 (316,),
 (315,),
 (314,),
 (313,),
 (312,),
 (311,),
 (310,),
 (309,),
 (308,),
 (307,),
 (306,),
 (305,),
 (304,),
 (303,),
 (302,),
 (301,),
 (300,),
 (291,),
 (290,),
 (289,),
 (288,),
 (287,),
 (286,),
 (285,),
 (284,),
 (283,),
 (282,),
 (281,),
 (280,),
 (279,),
 (278,),
 (277,),
 (276,),
 (275,),
 (274,),
 (273,),
 (272,),
 (271,),
 (270,),
 (269,),
 (268,),
 (267,),
 (266,),
 (265,),
 (264,),
 (263,),
 (262,),
 (261,),
 (260,),
 (259,),
 (258,),
 (257,),
 (256,),
 (255,),
 (254,),
 (253,),
 (252,),
 (251,),
 (250,),
 (243,),
 (242,),
 (240,),
 (236,),
 (235,),
 (234,),
 (233,),
 (232,),
 (231,),
 (230,),
 (229,),
 (228,),
 (227,),
 (226,),
 (225,),
 (224,),
 (223,),
 (222,),
 (221,),
 (220,),
 (219,),
 (218,),
 (217,),
 (216,),
 (215,),
 (214,),
 (213,),
 (212,),
 (211,),
 (210,),
 (209,),
 (208,),
 (207,),
 (206,),
 

In [138]:
scores = db.query("Select average_score from random_samples_average_score where size= '87'").getresult()
scores

[(0.0191894734954,),
 (0.0320208587831,),
 (0.0209792413512,),
 (0.0413694095903,),
 (0.0321846512187,),
 (0.0388805533642,),
 (0.0262993077276,),
 (0.0397196352812,),
 (0.0351052171959,),
 (0.0398817182783,),
 (0.0250257219762,),
 (0.0248957495878,),
 (0.0242848316552,),
 (0.0270109516599,),
 (0.0241490664781,),
 (0.0291314239437,),
 (0.021266935661,),
 (0.0301805654193,),
 (0.0362946049104,),
 (0.0313978719797,),
 (0.0159146359825,),
 (0.0401487952955,),
 (0.0329601326835,),
 (0.0283077385246,),
 (0.0185433577486,),
 (0.0402513680779,),
 (0.0306697569535,),
 (0.0328395023993,),
 (0.0325600844428,),
 (0.0210137576796,),
 (0.0328929954249,),
 (0.0321127461888,),
 (0.0290737970462,),
 (0.0269380248489,),
 (0.031802250528,),
 (0.0231196262053,),
 (0.0335096903812,),
 (0.0369668656761,),
 (0.0343856800493,),
 (0.031423228047,),
 (0.0208903809734,),
 (0.0301094127516,),
 (0.0260460293307,),
 (0.0194505701382,),
 (0.0199894711747,),
 (0.0324169230865,),
 (0.0207254511942,),
 (0.026058570035

In [54]:

for size_t in sample_average_score_sizes:
    size = size_t[0]
    print(size)
    motifs_of_size = db.query("SELECT motif_string, average_score, tt_average_score, proteins_length FROM motif_average_scores where average_score NOTNULL and proteins_length = '" + str(size) + "'").getresult()
    motifs_of_size_shuffled = db.query("SELECT motif_string, shuffled_average_score, tt_shuffled_average_score, shuffled_proteins_length FROM motif_average_scores where shuffled_average_score NOTNULL and shuffled_proteins_length = '" + str(size) + "'").getresult()
    for motif_t in motifs_of_size:
        print(motif_t)
        if(db.query("SELECT p_value FROM motif_average_scores where motif_string = '" + motif_t[0] +"' and p_value NOTNULL").ntuples() > 0):
            print("skipping real size", size)          
            continue
        size = motif_t[3]
        average_score = motif_t[1]
        tt_average_score = motif_t[2]
        total_distribution = db.query("Select count(average_score) from samples_scores where size= '" + str(size) +"'").getresult()[0][0]
        right_of_distribution = db.query("Select count(average_score) from samples_scores where size = '" + str(size) + "' and average_score > '" +str(average_score) +"'").getresult()[0][0]
        tt_right_of_distribution = db.query("Select count(average_score) from samples_scores where size = '" + str(size) + "' and tt_average_score > '" +str(tt_average_score) +"'").getresult()[0][0]

        p_value = right_of_distribution/total_distribution
        tt_p_value = tt_right_of_distribution/total_distribution        
#         print("real", size, average_score ,motif_t[0], p_value)
        do_not_print = db.query("UPDATE motif_average_scores SET p_value = " + str(p_value) + ",tt_p_value = " + str(tt_p_value) + "  WHERE motif_string  = '" + motif_t[0] + "'")

    for motif_shuffled_t in motifs_of_size_shuffled:
        print(motif_shuffled_t)
        if(db.query("SELECT shuffled_p_value FROM motif_average_scores where motif_string = '" + motif_shuffled_t[0] +"' and shuffled_p_value NOTNULL").ntuples() > 0):
            print("skipping shuffled size", size)
            continue
        shuffled_size = motif_shuffled_t[3]
        shuffled_average_score = motif_shuffled_t[1]
        tt_shuffled_average_score = motif_shuffled_t[2]
        total_distribution = db.query("Select count(average_score) from samples_scores where size= '" + str(shuffled_size) +"'").getresult()[0][0]
        right_of_distribution = db.query("Select count(average_score) from samples_scores where size = '" + str(shuffled_size) + "' and average_score > '" +str(shuffled_average_score) +"'").getresult()[0][0]
        tt_right_of_distribution = db.query("Select count(average_score) from samples_scores where size = '" + str(shuffled_size) + "' and tt_average_score > '" +str(tt_shuffled_average_score) +"'").getresult()[0][0]
        shuffled_p_value = right_of_distribution/total_distribution
        tt_shuffled_p_value = tt_right_of_distribution/total_distribution

#         print("shuff", shuffled_size, shuffled_average_score,motif_shuffled_t[0], shuffled_p_value)
        do_not_print = db.query("UPDATE motif_average_scores SET shuffled_p_value = '" + str(shuffled_p_value) + "', tt_shuffled_p_value = '" + str(tt_shuffled_p_value) + "'WHERE motif_string  = '" + motif_shuffled_t[0] + "'")





334
('aat[ag]tgct', 0.0330842760542, 0.223502436574, 334)
skipping real size 334
('at[ag]gat[ct]a', 0.0312657622814, 0.211637489875, 334)
skipping real size 334
('aactgcc.', 0.0302644556558, 0.219819915089, 334)
skipping real size 334
('aagtggc.', 0.0290509311628, 0.220138264242, 334)
skipping real size 334
('ac[ag].ctat', 0.0309410804744, 0.224828186197, 334)
skipping real size 334
('acgt.t[ct]t', 0.0286139837129, 0.225195531788, 334)
skipping real size 334
('aaacata[ag]', 0.0313143303906, 0.217242495137, 334)
skipping real size 334
('aaagata[ct]', 0.0313374686414, 0.215882683412, 334)
skipping real size 334
('aaagc[ag]c[ct]', 0.0304390620578, 0.207817036351, 334)
skipping real size 334
('aaatc[ct]ta', 0.0293729051206, 0.213946663315, 334)
skipping real size 334
('aaatggt[ct]', 0.0317686244289, 0.216041171113, 334)
skipping real size 334
('aat[ag]tcat', 0.031036318062, 0.225148552063, 334)
skipping real size 334
('aat[ct]g[ag]at', 0.0337990176949, 0.216335816688, 334)
skipping real si

('a[ag][ct]gcaat', 0.0277872913533, 0.213037273763, 334)
('a[ag].cggcg', 0.0330339314555, 0.215388584972, 334)
('aaa[ct]tc[ct]g', 0.0286019290633, 0.210510444229, 334)
skipping shuffled size 334
('acaa[ag]ct[ct]', 0.0311797951195, 0.218650222125, 334)
skipping shuffled size 334
('aca[ag]t[ct]ag', 0.0317393232639, 0.220298080429, 334)
skipping shuffled size 334
('aca[ag][ag]tac', 0.0297262674235, 0.208991449583, 334)
skipping shuffled size 334
('aca[ct].gcg', 0.0299628426648, 0.209314760979, 334)
skipping shuffled size 334
('aca[ct]ca[ct]t', 0.0338738887033, 0.208305732186, 334)
skipping shuffled size 334
('aca[ct]aa[ct]g', 0.0313538247644, 0.20564144544, 334)
skipping shuffled size 334
('aca[ct]gt[ag]t', 0.0288432269195, 0.215778885025, 334)
skipping shuffled size 334
('aca[ct]ct[ag]t', 0.0330991843624, 0.222559727966, 334)
skipping shuffled size 334
('accct[ag]t[ct]', 0.0280458898783, 0.21642741838, 334)
skipping shuffled size 334
('acccc.ac', 0.0305537028721, 0.207071515376, 334)
ski

('aaac[ag][ct]gt', 0.0265937237693, 0.212607851913, 333)
('aaac[ag]gg[ag]', 0.032088593149, 0.212613044612, 333)
('aaac[ct]c[ct]c', 0.0309102164496, 0.218279820999, 333)
('aaacagaa', 0.0305343858785, 0.223105243904, 333)
('aaacg[ct][ag].', 0.0291299312084, 0.207391950619, 333)
('aaagaac[ct]', 0.0290905373196, 0.229767015778, 333)
('aaagatc.', 0.0254393162723, 0.212030082643, 333)
('aaag.cg.', 0.0290952993851, 0.214229861086, 333)
('aaataa[ct]c', 0.0229332279926, 0.209768558622, 333)
('aaatc[ag]c[ct]', 0.0323395694319, 0.21624368518, 333)
('aat[ag]acat', 0.0309444141884, 0.213473289859, 333)
('aat[ag]ggt[ag]', 0.0309467477505, 0.231554405578, 333)
('aatt[ct]aag', 0.0304419558983, 0.214214611409, 333)
('aattcaa[ag]', 0.032094787964, 0.214830288418, 333)
('aatttg[ag]g', 0.0304794184991, 0.218943319365, 333)
('a[ag].ggcg[ct]', 0.0266081869146, 0.20356543508, 333)
('a[ag].gtacc', 0.0281111204723, 0.203417981826, 333)
('a[ct]a[ct]c[ag]ta', 0.0283374867271, 0.223159257253, 333)
('a[ct]a[ct]ca

('ag[ct]aag[ag]t', 0.0369635959033, 0.217305883625, 333)
('ag[ct]acct[ag]', 0.0292476258338, 0.226840718932, 333)
('ata.gc[ct]g', 0.0327324975269, 0.208713332887, 333)
('ag[ct][ct]taca', 0.0307646271908, 0.209418234795, 333)
('ag.accaa', 0.0312752285287, 0.224576823053, 333)
('ag.acttc', 0.0307678428012, 0.216747186937, 333)
('ata.tcca', 0.0254873921057, 0.211305406448, 333)
('atg.ccat', 0.0280033565574, 0.207697522258, 333)
('ata[ag]t[ct]gt', 0.031754609482, 0.222306048825, 333)
('ata[ct][ag][ag]cc', 0.02531624322, 0.207115246912, 333)
('atg.catc', 0.0313004774237, 0.22485260613, 333)
('ata[ag]gtc.', 0.0283410469024, 0.220799020008, 333)
('ata[ct]gc.a', 0.0302779094351, 0.208099290797, 333)
('atg[ct][ag]tgc', 0.0312856655963, 0.219631819914, 333)
('atg[ct][ag]gca', 0.0299189667735, 0.218297292471, 333)
('at[ag]agg[ct]t', 0.035742022328, 0.219247520814, 333)
('aagt.tcc', 0.0256315637776, 0.213246488778, 333)
('att.[ag]cg.', 0.031508775265, 0.219583012056, 333)
('aag[ct]caa[ct]', 0.0355

('aa[ct]cgc.g', 0.0292699219723, 0.20060102322, 333)
('aa[ct][ct]agtc', 0.0298482910521, 0.220426481372, 333)
('aa[ct]ctag[ct]', 0.0287082326013, 0.210775205413, 333)
('aa[ct][ct]agtg', 0.0306552168981, 0.206740663539, 333)
('aa[ct]gag[ag]g', 0.0291754297886, 0.20405217443, 333)
('aa[ct]tc[ag]ga', 0.0302477155549, 0.2133542935, 333)
('aa[ct][ag]ctct', 0.0278046812609, 0.209102400517, 333)
('aacaa.gg', 0.031547300302, 0.210704808448, 333)
('ca.[ct]cgga', 0.0339885955541, 0.207702149896, 333)
('aa.gggtg', 0.028811018709, 0.213966880621, 333)
('acaattt[ag]', 0.0338514539586, 0.220938116498, 333)
('acaat[ct]c[ct]', 0.0291067519642, 0.197702606939, 333)
('aca.cg[ct]g', 0.0353895348364, 0.223326769902, 333)
('aca[ag][ct]gat', 0.0319679116938, 0.221962905404, 333)
('aca[ag]c[ag]ta', 0.0261556381592, 0.212379837518, 333)
('aca[ct]atg[ct]', 0.0281683450056, 0.21023999142, 333)
('aca[ag]ttg[ag]', 0.0272568495475, 0.210962295212, 333)
('accata[ct][ct]', 0.0238723313549, 0.211434446001, 333)
('acg

...

('aca[ag][ag]tac', 0.0305011955405, 0.226321396978, 304)
('agaaac[ag]a', 0.0321509635169, 0.217771369179, 304)
('aa[ct]ataca', 0.0260454405548, 0.210678913151, 304)
('aa[ct]c[ct]ctc', 0.0347799957023, 0.223148065116, 304)
('aaaac[ct]ac', 0.0301976104168, 0.206555178815, 304)
('aacttc.c', 0.0284701794792, 0.204678763791, 304)
('aa[ag]c[ag]gta', 0.0303253336174, 0.21102515445, 304)
('aaaagg[ag]t', 0.0293336092403, 0.223065578873, 304)
('aaacc[ag]c[ct]', 0.0276494809719, 0.205250553399, 304)
('aaacatc[ct]', 0.0275107215825, 0.212418399559, 304)
('aaact[ct]tc', 0.0284696286256, 0.218338508002, 304)
('aattag[ct]c', 0.0294566209869, 0.221572538473, 304)
('aaat.ggc', 0.0259101195246, 0.213598547689, 304)
('aaatcatt', 0.0358059282445, 0.219913138049, 304)
('aaatgc[ct]g', 0.0271774560138, 0.207074292774, 304)
('aat[ag]cc[ag]a', 0.0304113430556, 0.211840836914, 304)
('aat[ag]tcaa', 0.0321612875722, 0.217739699709, 304)
('aatt[ag][ag]tc', 0.0307644927482, 0.214322025024, 304)
('aaactct[ag]', 0.03

('a[ag]ac[ag]agg', 0.0290937461483, 0.213917116921, 304)
('at[ct]gcaa[ct]', 0.0280521101767, 0.208708233148, 304)
('at[ct]gg[ag]ag', 0.0296830387284, 0.210291784579, 304)
('at[ct][ag]tcac', 0.0294686452133, 0.234198918673, 304)
('aag[ct]t[ct]cc', 0.0298988813689, 0.209310194797, 304)
('at[ct][ag]agca', 0.0299286283858, 0.209099052222, 304)
('at[ct][ag]aggg', 0.0256568507635, 0.213889640091, 304)
('at[ct][ag]cagt', 0.0309684432922, 0.215012904329, 304)
('at[ct][ct]aacc', 0.0287481649584, 0.222257162602, 304)
('at[ct]tcc[ct]g', 0.0334038966296, 0.227330096212, 304)
('at[ct]ttgtc', 0.0314135339058, 0.216530867411, 304)
('aag[ct]tca[ct]', 0.0295045602476, 0.224023937296, 304)
('a[ag]accat[ag]', 0.0273109571885, 0.211240424067, 304)
('a[ag]agtaag', 0.0364347118113, 0.21349075877, 304)
('a[ag]agagtt', 0.0298675481875, 0.232966808223, 304)
('a[ag]ag[ag]cag', 0.0296650134782, 0.210219296622, 304)
('a[ag]a[ag]gcag', 0.0301542534277, 0.217387689504, 304)
('a[ag]atcac[ct]', 0.0317126035682, 0.218

('aac[ag]ac[ag][ag]', 0.0312359992183, 0.215535989515, 303)
('aac[ag]at[ag]t', 0.0327661133973, 0.204978382478, 303)
('aac[ag]tttc', 0.0338106338792, 0.218078031568, 303)
('ac.[ct][ct]ccg', 0.0303025903286, 0.212311535333, 303)
('aactcc.a', 0.0311955662406, 0.210099067347, 303)
('agaag[ag]gg', 0.0322648979042, 0.237239735951, 303)
('ac.[ct]gcaa', 0.0295561433114, 0.228219074523, 303)
('agtag[ag]a[ct]', 0.0304075203819, 0.215663913316, 303)
('aga.accc', 0.0339938470075, 0.216141251409, 303)
('agca[ag]ac[ag]', 0.0274597591164, 0.209547215122, 303)
('agca[ag]att', 0.0237830618806, 0.216106794776, 303)
('agcaa[ag]c[ag]', 0.0246677659024, 0.204241479378, 303)
('agg.ccg.', 0.0272481413251, 0.218722316872, 303)
('aac[ct][ct]tag', 0.0306067077661, 0.22308836493, 303)
('agtcacc.', 0.0285919168417, 0.209414254802, 303)
('aac[ct]tat[ag]', 0.0266315982042, 0.228773393937, 303)
('aac[ct]ca[ag]c', 0.0306605487211, 0.207792674808, 303)
('aagactc.', 0.0288967148156, 0.209835534653, 303)
('aagaa[ct]ca'

('aa[ag]gtc[ct]g', 0.0312854221801, 0.209002264324, 303)
('a.aa[ag][ct]cg', 0.0262466304926, 0.213659680679, 303)
('aa[ag]gtg[ct]g', 0.0279455316113, 0.209856366801, 303)
('a.agcacc', 0.0291236784968, 0.221163942986, 303)
('a[ct]..tacg', 0.0296373990593, 0.216804271147, 303)
('a.acgg[ag]g', 0.0358610545965, 0.226884014124, 303)
('a.a[ct]gcg.', 0.0315444953739, 0.206177423196, 303)
('a.c.cacg', 0.0328053470518, 0.210468130215, 303)
('a.c[ag]c[ag]ac', 0.029092915801, 0.211698302201, 303)
('a.g[ag]tacc', 0.0268483746158, 0.204432418001, 303)
('a.[ag]cg[ct]c[ag]', 0.0278267977749, 0.221961693709, 303)
('a.g[ct]gtag', 0.0300864079149, 0.21672929839, 303)
('a.tagcta', 0.0335615390696, 0.22456314758, 303)
('a.tgcacc', 0.0273890177438, 0.215672170386, 303)
('a.[ag].gtcg', 0.031489285508, 0.216923251303, 303)
('a.[ct]ggccg', 0.027399015933, 0.197545564289, 303)
('aa[ct]a[ct]tgc', 0.0325745657383, 0.209361146117, 303)
('aa[ct]at[ct]cc', 0.0286262666363, 0.210192812593, 303)
('aa[ct]c[ct]aac', 0.

KeyboardInterrupt: 

In [22]:
for motif_t in motifs_of_size:
    size = motif_t[2]
    average_score = motif_t[1]
    total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(size) +"'").getresult()[0][0]
    right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(size) + "' and average_score > '" +str(average_score) +"'").getresult()[0][0]
    p_value = right_of_distribution/total_distribution
    print(motif_t[0], p_value)



In [24]:
for motif_shuffled in motifs_of_size_shuffled:
    shuffled_size = motif_shuffled_t[3]
    shuffled_average_score = motif_shuffled[1]
    total_distribution = db.query("Select count(average_score) from random_samples_average_score where size= '" + str(shuffled_size) +"'").getresult()[0][0]
    right_of_distribution = db.query("Select count(average_score) from random_samples_average_score where size = '" + str(shuffled_size) + "' and average_score > '" +str(shuffled_average_score) +"'").getresult()[0][0]
    shuffled_p_value = right_of_distribution/total_distribution
    print(motif_shuffled[0], shuffled_p_value)




In [21]:
motif_shuffled_t

('ttttacta', 0.0304433909754, 0.226950680281, 305)

In [58]:
db.query("ALTER TABLE motif_average_scores ADD COLUMN p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN shuffled_p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN tt_p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN tt_shuffled_p_value float;")

In [57]:
# db.query("ALTER TABLE motif_average_scores drop COLUMN p_value ;")
# db.query("ALTER TABLE motif_average_scores drop COLUMN shuffled_p_value ;")
# db.query("ALTER TABLE motif_average_scores drop COLUMN tt_p_value ;")
# db.query("ALTER TABLE motif_average_scores drop COLUMN tt_shuffled_p_value ;")

In [ ]:
db.query("Select count(*) from motif_average_scores ").getresult()


In [119]:
total_distribution

10002

In [120]:
right_of_distribution


603

In [6]:
p_value = right_of_distribution/total_distribution




ZeroDivisionError: division by zero

In [40]:
q = db.query("SELECT * FROM motif_average_scores")
print(q)

motif_string|proteins_length| average_score |tt_average_score|shuffled_proteins_length|shuffled_average_score|tt_shuffled_average_score
------------+---------------+---------------+----------------+------------------------+----------------------+-------------------------
aaaaacc.    |            616|0.0266036989842|   0.20606186622|                     514|       0.0292231008496|           0.206730352948
aaaaacc[ag] |            312|0.0320499827315|  0.219748374339|                     283|       0.0308457989402|           0.211855552231
aaaaacc[ct] |            371|0.0236889137655|  0.202720019551|                     270|       0.0314594132445|           0.215861952133
aaaaacga    |             46|0.0425184731154|  0.260826692549|                     195|       0.0286965889542|           0.232272194289
aaaaacgc    |             27| 0.061532254768|   0.34081391517|                      98|       0.0341177459883|           0.240031032202
aaaaacgg    |             38|0.0532548463267|   